In [1]:
import tensorflow as tf
import numpy as np
import gym
from tensorflow import keras
import math
import random

In [2]:
# env = gym.make('BipedalWalker-v3')
# #reset inizializza environment
# #observation contiene le informazioni relative allo stato
# observation = env.reset()
# print("Observation: ", observation)
# #numero di azioni che è possibile eseguire
# print("Action space: ",env.action_space)
# #numero di osservazioni che otteniamo dallo stato,
# #ad esempio nel primo stato otteniamo quelle stampate precedentemente
# print("Observation: ", env.observation_space)

Volgio utilizzare una DQN per addestrare il mio agente in grado di imparare a camminare. Per questo metodo avrò bisogno:
1. Epsilon greedy selection rule: per selezionare l'azione, con probabilità epsilon andrò a selezionare un'azione random, nel resto dei casi l'azione sarà calcolata da una Neural Network.
2. Neural Network: la rete avrà un input layer con 160 nodi (il numero di observation fornite dalo state) e nell'output layer vi saranno 4 nodi, pari al numero di azioni da eseguire, per ogni nodo vi sarà un valore che rappresenta il Q-value, e noi saremo interessati al valore massimo.
3. Nel caso della DQN, per eseguire l'update dei parametri della rete, (Weight of the connections), andremo ad utilizzare una experience replay.
4. Experience Replay: non è altro che un batch in cui andramo ad inserire tutti gli stati visitati con le relative informazioni fornite da env.step(action)
5. Per eseguire l'update andremo, dopo un certo numero di timestep ad eseguire l'update tramite il policy gradient method.
6. Devo discretizzare l'uscita, ogni nodo corrisponde a 0.1 (-1, -0.9, .. 1 per la prima azione e così via)

In [3]:
NUM_EPISODE = 1000
NUM_TIMESTEP = 400
LEARNING_RATE = 0.001 
DISCOUNT_FACTOR = 0.9
BATCH_SIZE = 32 #from the implementation of DQN for Atari games

environment = gym.make('BipedalWalker-v3')
#retrieve possible action that I can apply
INPUT = [environment.observation_space.shape[0]]
n_outputs = [environment.action_space.shape[0]]

optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_function = keras.losses.mean_squared_error

model = keras.Sequential([
    keras.layers.Dense(50,activation="relu", input_shape=INPUT),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(21*n_outputs[0])
])

#model.compile(optimizer=optimizer, loss=loss_function)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1250      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 84)                4284      
Total params: 8,084
Trainable params: 8,084
Non-trainable params: 0
_________________________________________________________________


c:\users\perro\appdata\local\programs\python\python38\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
dictionarie =  {}
start =  - 1.0
for i in range(84):
    dictionarie[i] = start
    if start == 1:
        start = -1
    else:
        start = round(start + 0.1,1)
print(dictionarie)

list_value = [-1.0, -0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
print(list_value)

{0: -1.0, 1: -0.9, 2: -0.8, 3: -0.7, 4: -0.6, 5: -0.5, 6: -0.4, 7: -0.3, 8: -0.2, 9: -0.1, 10: 0.0, 11: 0.1, 12: 0.2, 13: 0.3, 14: 0.4, 15: 0.5, 16: 0.6, 17: 0.7, 18: 0.8, 19: 0.9, 20: 1.0, 21: -1, 22: -0.9, 23: -0.8, 24: -0.7, 25: -0.6, 26: -0.5, 27: -0.4, 28: -0.3, 29: -0.2, 30: -0.1, 31: 0.0, 32: 0.1, 33: 0.2, 34: 0.3, 35: 0.4, 36: 0.5, 37: 0.6, 38: 0.7, 39: 0.8, 40: 0.9, 41: 1.0, 42: -1, 43: -0.9, 44: -0.8, 45: -0.7, 46: -0.6, 47: -0.5, 48: -0.4, 49: -0.3, 50: -0.2, 51: -0.1, 52: 0.0, 53: 0.1, 54: 0.2, 55: 0.3, 56: 0.4, 57: 0.5, 58: 0.6, 59: 0.7, 60: 0.8, 61: 0.9, 62: 1.0, 63: -1, 64: -0.9, 65: -0.8, 66: -0.7, 67: -0.6, 68: -0.5, 69: -0.4, 70: -0.3, 71: -0.2, 72: -0.1, 73: 0.0, 74: 0.1, 75: 0.2, 76: 0.3, 77: 0.4, 78: 0.5, 79: 0.6, 80: 0.7, 81: 0.8, 82: 0.9, 83: 1.0}
[-1.0, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


In [5]:
def get_max_Q_values_each_action(Q_values):
    best_index_action = []
    for i in range(4):
        #print("Range: from {} to {} ".format(21*i, 21*(i+1)) )
        best_index_action.append(np.argmax(Q_values[0][21*i:21*(i+1)]) + 21*i)
        #print(best_index_action)
    return np.array(best_index_action, dtype=np.int16)

In [6]:
#I use DQN so I need Q-learning and Neural Network
#For Q-learning I use epsilon greedy seletion rule, I need a function to do that
def epsilon_greedy(state, epsilon):
    action = [0,0,0,0]
    index  = [0,0,0,0]
    if np.random.rand() < epsilon:
        #I pick a random action
        for i in range(4):
            action[i] = random.choice(list_value)
            #print("Action = ",action)
            for k,v in dictionarie.items():
                if v == action[i]:
                    index[i] = k + 21*i
                    #print("Index ", index)
                    break
        return action, index
    else:
        #Starting from the state and using the network I generate the qvalues
        #associated to the action and pick the highest
        Q_values = model.predict(state[np.newaxis])
        #print(Q_values)
        index_max_qvalues = get_max_Q_values_each_action(Q_values)
        #varie alternative
        for i in range(4):
            action[i] = dictionarie[index_max_qvalues[i]]
        #print("Action ", action)
        return action, index_max_qvalues

In [7]:
def perform_action(observation, epsilon, environment):
    #1. seleziono azione con epsilon greedy policy
    action, index = epsilon_greedy(observation, epsilon)
    state, reward, done, info = environment.step(action)
    #2. aggiungo nel replpay buffer
    experience_replay.append([observation, reward, done, index, state])
    return state, reward, done, info

In [8]:
experience_replay = []

In [9]:
def get_experience():
    random_index = np.random.randint(low=0, high=len(experience_replay), size=BATCH_SIZE)
    #recupero dall'experience replay i vettori corrispondenti agli indici random
    batch= [experience_replay[index] for index in random_index]
    batch_state, batch_reward, batch_done, batch_action_index, batch_next_state = [np.array([batch_element[field] for batch_element in batch]) for field in range(5)]
    return batch_state, batch_reward, batch_done, batch_action_index, batch_next_state


In [10]:
def get_batch_max_Q_values_each_action(Q_values):
    max1, max2, max3,max4 = [],[],[],[]
    for q in Q_values:
        max1.append(np.max(q[0:22]))
        max2.append(np.max(q[21:42]))
        max3.append(np.max(q[41:63]))
        max4.append(np.max(q[62:84]))
    
    max1 = np.array(max1)
    max2 = np.array(max2)
    max3 = np.array(max3)
    max4 = np.array(max4)
    Q_next = np.array([max1, max2,max3,max4])
    return Q_next

In [11]:
def batch_index_single_action(batch, action):
    index_first_action = np.empty((32,1), dtype=np.int16)
    for i in range(32):
        index_first_action[i] = batch[i][action]
    return index_first_action.transpose()

In [12]:
def update_network():
    exp = get_experience()
    batch_state, batch_reward, batch_done, batch_action_index, batch_next_state = exp
    #Q LEARNING FORMULA
    #Q_t = Q_t + alpha(Rt+1 + gamma(maxQt+1) - Q_t)
    #I need Q_t+1
    Q_next_state = model.predict(batch_next_state)
    Q_next_state_per_action = get_batch_max_Q_values_each_action(Q_next_state) #4 vettori da 32 elementi
    delta = []
    loss = []
    for i in range(4):
        delta.append(batch_reward + (1 - batch_done)*DISCOUNT_FACTOR*(Q_next_state_per_action[i]))
    
    delta = np.array(delta)
    #print("shape", delta.shape) #4,32
    index_first_action = batch_index_single_action(batch_action_index, 0)
    index_second_action = batch_index_single_action(batch_action_index,1)
    index_third_action = batch_index_single_action(batch_action_index,2)
    index_fourth_action = batch_index_single_action(batch_action_index,3)
    tot_index =  np.array([index_first_action, index_second_action, index_third_action, index_fourth_action])
    #print(tot_index)
    #print("shape tot_index ", tot_index.shape)
    tot_index = tot_index.reshape(4,32)
    #print(tot_index)
    #print("shape tot_index ", tot_index.shape)
    with tf.GradientTape(persistent=True) as tape:
        all_Q_values = model(batch_state)
        for i in range(4):
            mask = tf.one_hot(tot_index[0][i], 21*n_outputs[0])
            #print("Mask: ",mask)
            Q_values = tf.reduce_sum(all_Q_values*mask, axis=1, keepdims=True)
            loss.append(tf.reduce_mean(loss_function(delta[0][i],Q_values)))
    
    #print("Loss",loss)
    for i in range(4):
        grads = tape.gradient(loss[i], model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
done_external = False

for episode in range(NUM_EPISODE):
    cumulative_reward = 0
    print("Current episode: ",episode)
    state = environment.reset()
    for timestep in range(NUM_TIMESTEP):
        epsilon = max(1 - episode/200, 0.1)
        state, reward, done, info = perform_action(state, epsilon, environment)
        cumulative_reward += reward
        if reward == -100: #il robot è caduto
            break;
        if done:
            print("Game solved!!")
            done_external = done
            break
        if episode > 3: #per avere un numero di elementi maggiore della dimensione del batchsize
            update_network()
        environment.render()
        
    if done_external:
        break
    print("Cumulative Reward episode {} = {} ".format(episode, cumulative_reward))
    print("Episode {} completed.".format(episode))

Current episode:  0
Cumulative Reward episode 0 = -18.954857562740383 
Episode 0 completed.
Current episode:  1
Cumulative Reward episode 1 = -16.888183726662252 
Episode 1 completed.
Current episode:  2
Cumulative Reward episode 2 = -103.18986475681129 
Episode 2 completed.
Current episode:  3
Cumulative Reward episode 3 = -109.99622193980242 
Episode 3 completed.
Current episode:  4
Cumulative Reward episode 4 = -101.6607135113541 
Episode 4 completed.
Current episode:  5
Cumulative Reward episode 5 = -24.133149330383542 
Episode 5 completed.
Current episode:  6
Cumulative Reward episode 6 = -99.9153119497329 
Episode 6 completed.
Current episode:  7
Cumulative Reward episode 7 = -22.81874897988725 
Episode 7 completed.
Current episode:  8
Cumulative Reward episode 8 = -112.2652514123431 
Episode 8 completed.
Current episode:  9
Cumulative Reward episode 9 = -109.51657390672912 
Episode 9 completed.
Current episode:  10
Cumulative Reward episode 10 = -99.64583477690121 
Episode 10 co